In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from collections import Counter
import os
import sys
import re
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, ReLU, Dropout, Flatten, MaxPool2D
from keras.utils import to_categorical
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
tqdm.pandas()
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 150)
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']

In [4]:
filePath = "data/tinder.xlsx"
imagePath = "data/photos"

In [5]:
df = pd.read_excel(filePath)
df.drop_duplicates(inplace=True, subset="id")
df.set_index("id", inplace=True)
df["bio"] = df["bio"].fillna("")

In [6]:
X=[]
y=[]
for fileName in tqdm(os.listdir(imagePath)):
    try:
        id_ = re.match("([a-z0-9]*)-\d( \(\d\))?.jpg",fileName).group(1)
        match = df.loc[id_]["match"]
        filePath = os.path.join(imagePath, fileName)
        img = cv2.imread(filePath)
        img = cv2.resize(img, (120,120))
        X.append(img)
        y.append(match)
    except:
        pass
X=np.asarray(X)
y=np.asarray(y)

In [7]:
X=(X/255).astype(np.float16)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8888)

In [9]:
def getModel():
    model=Sequential()
    model.add(Conv2D(3,3,input_shape=(120,120,3)))
    model.add(ReLU())
    model.add(MaxPool2D((2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(3,3,padding="same"))
    model.add(ReLU())
    model.add(MaxPool2D((2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Dense(2,activation="softmax"))
    return model

In [10]:
model = getModel()

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 118, 118, 3)       84        
_________________________________________________________________
re_lu_1 (ReLU)               (None, 118, 118, 3)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 59, 59, 3)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 59, 59, 3)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 59, 59, 3)         84        
_________________________________________________________________
re_lu_2 (ReLU)               (None, 59, 59, 3)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 29, 29, 3)        

In [12]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=["accuracy"])

In [13]:
model.fit(X_train, to_categorical(y_train), epochs=30, validation_data=(X_test, to_categorical(y_test)))

Train on 19473 samples, validate on 6491 samples
Epoch 1/30
19473/19473 [==============================] - 160s 8ms/step - loss: 0.0556 - accuracy: 0.9929 - val_loss: 0.0763 - val_accuracy: 0.9915
Epoch 2/30
19473/19473 [==============================] - 134s 7ms/step - loss: 0.0433 - accuracy: 0.9931 - val_loss: 0.0547 - val_accuracy: 0.9915
Epoch 3/30
19473/19473 [==============================] - 132s 7ms/step - loss: 0.0424 - accuracy: 0.9930 - val_loss: 0.0566 - val_accuracy: 0.9915
Epoch 4/30
19473/19473 [==============================] - 142s 7ms/step - loss: 0.0393 - accuracy: 0.9929 - val_loss: 0.0511 - val_accuracy: 0.9915
Epoch 5/30
19473/19473 [==============================] - 131s 7ms/step - loss: 0.0354 - accuracy: 0.9930 - val_loss: 0.0528 - val_accuracy: 0.9915
Epoch 6/30
19473/19473 [==============================] - 131s 7ms/step - loss: 0.0350 - accuracy: 0.9930 - val_loss: 0.0512 - val_accuracy: 0.9915
Epoch 7/30
19473/19473 [==============================] - 137s 

In [14]:
y_pred=model.predict(X_test)

In [15]:
y_pred=np.exp(y_pred)
y_pred=(y_pred/np.sum(y_pred,axis=1).reshape(-1,1))[:,1]

In [16]:
roc_auc_score(y_test,y_pred)

0.5116927510028815